### 1. 데이터 불러오기

In [1]:
import pandas as pd

df1 = pd.read_csv("../data/review-rating1.csv")
df2 = pd.read_csv("../data/review-rating2.csv")
df3 = pd.read_csv("../data/review-rating3.csv")
df4 = pd.read_csv("../data/review-rating4.csv")

df = pd.concat([df1, df2, df3, df4])
df.head()

,실제평점,리뷰,영화ID,영화명,평균평점
0,10.0,이 영화의 속편은 100% 성공이다. 원래 한국영화들 속편은 잘 안되고 속편 나오면...,0,범죄도시2,8.99
1,10.0,진짜 재미있어요잔인한 거 잘 못 보는데 충분히 볼 수 있구요. 유머도 내 스타일이라...,0,범죄도시2,8.99
2,10.0,넘 재밌어요 시원시원하고 계속 시리즈로 나왔으면 좋겠어요. 나쁜놈들 잡는 캐릭터로...,0,범죄도시2,8.99
3,10.0,솔직하게 장첸보다 강해상이 더 강해보인다,0,범죄도시2,8.99
4,10.0,범죄도시 1편도 2편도 다 악역배우 연기가 진짜 미쳤다 ㅠㅠㅠ 마동석 배우 액션도 ...,0,범죄도시2,8.99


### 2. 전처리

In [2]:
# 특수문자,기호 제거
df['리뷰'] = df['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df[:5]

C:\Users\gram15\AppData\Local\Temp\ipykernel_8924\2647383859.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['리뷰'] = df['리뷰'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,실제평점,리뷰,영화ID,영화명,평균평점
0,10.0,이 영화의 속편은 성공이다 원래 한국영화들 속편은 잘 안되고 속편 나오면 망하는 ...,0,범죄도시2,8.99
1,10.0,진짜 재미있어요잔인한 거 잘 못 보는데 충분히 볼 수 있구요 유머도 내 스타일이라 ...,0,범죄도시2,8.99
2,10.0,넘 재밌어요 시원시원하고 계속 시리즈로 나왔으면 좋겠어요 나쁜놈들 잡는 캐릭터로는...,0,범죄도시2,8.99
3,10.0,솔직하게 장첸보다 강해상이 더 강해보인다,0,범죄도시2,8.99
4,10.0,범죄도시 편도 편도 다 악역배우 연기가 진짜 미쳤다 ㅠㅠㅠ 마동석 배우 액션도 진짜...,0,범죄도시2,8.99


In [3]:
# 공백 제거
import numpy as np

df['리뷰'] = df['리뷰'].str.replace('^ +', "")
df['리뷰'].replace('', np.nan, inplace=True)
df.isnull().sum()

C:\Users\gram15\AppData\Local\Temp\ipykernel_8924\4057912268.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['리뷰'] = df['리뷰'].str.replace('^ +', "")


실제평점       0
리뷰      6690
영화ID       0
영화명        0
평균평점       0
dtype: int64

In [4]:
# null 샘플 제거
df = df.dropna(how = 'any')
len(df)

1177159

In [5]:
# 불용어 사전
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

In [6]:
# 불용어 추가
add_stopwords = ['ㅠ', 'ㅠㅠ', 'ㅠㅠㅠ', 'ㅋ', 'ㅋㅋ', 'ㅋㅋㅋ', '수', '편', '더', '오', '손', '구']
for word in add_stopwords:
    stopwords.append(word)

In [11]:
from tqdm import tqdm
from konlpy.tag import Okt
from wordcloud import WordCloud

okt = Okt()
df = df['리뷰']
word_list = []

for sentence in tqdm(df):
    words = okt.nouns(sentence)
    words = [word for word in words if not word in stopwords]
    word_list.append(words)

100%|██████████| 1177159/1177159 [1:17:52<00:00, 251.93it/s]


In [16]:
word_lis = []

for words in tqdm(word_list):
    for w in words:
        word_lis.append(w)

100%|██████████| 1177159/1177159 [00:00<00:00, 1448782.17it/s]


In [19]:
print('단어수 : ', len(word_lis))
word_set = set(word_lis)
print('단어수 : ', len(word_set))

단어수 :  8729593
단어수 :  95272


In [37]:
# 단어목록 중간저장
import csv
with open('../data/word_set.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(word_set)

with open('../data/word_lis.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(word_lis)

In [47]:
# 불러오기
import csv

with open('../data/word_set.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    word_set2 = list(reader)

with open('../data/word_lis.csv', 'r', newline='') as f:
    reader = csv.reader(f)
    word_lis2 = list(reader)

# 첫 번째 내부 리스트를 가져옴
word_set = word_set2[0]

# 두 번째 내부 리스트를 가져옴
word_lis = word_lis2[0]

In [48]:
tag_dict = {}

for item in tqdm(word_set):
  tag_dict[item] = word_lis.count(item)

# tag_dict = {item:tag_list.count(item) for item in tag_set}
tag_dict

 28%|██▊       | 26373/95272 [58:12<3:24:52,  5.60it/s]

In [ ]:
count = 0
for key, val in tag_dict.items():
  print(key, ':', val)
  count += 1
  if count > 5:
    break

### 3. 워드클라우드 만들기

In [ ]:
from wordcloud import WordCloud

font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

wc = WordCloud(background_color = 'white', random_state = 2023, font_path = font_path)
wc = wc.generate_from_frequencies(tag_dict)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10,6))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np

# 그림 위에 표시
mask = np.array(Image.open('../data/camera_icon.png'))
plt.imshow(mask, cmap = plt.cm.gray, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
wc = WordCloud(background_color = 'white', mask = mask, max_words = 1000, font_path = font_path).generate(tag_dict)
plt.figure(figsize = (12,8))     
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.show()